In [1]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName('Abhijit').getOrCreate()

##Loading Data

In [3]:
bfData = sc.textFile('dbfs:/FileStore/tables/BlackFriday.csv')
bfData.cache()
bfData.count()

Out[ 26 ]: 537578

In [4]:
bfData.take(5)

Out[ 27 ]: 
['User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase',
 '1000001,P00069042,F,0-17,10,A,2,0,3,,,8370',
 '1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200',
 '1000001,P00087842,F,0-17,10,A,2,0,12,,,1422',
 '1000001,P00085442,F,0-17,10,A,2,0,12,14,,1057']

## Filtering first line out.

In [6]:
dataLines = bfData.filter(lambda x: "User_ID" not in x)
dataLines.take(5)

Out[ 28 ]: 
['1000001,P00069042,F,0-17,10,A,2,0,3,,,8370',
 '1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200',
 '1000001,P00087842,F,0-17,10,A,2,0,12,,,1422',
 '1000001,P00085442,F,0-17,10,A,2,0,12,14,,1057',
 '1000002,P00285442,M,55+,16,C,4+,0,8,,,7969']

### Function to convert null values to 0 in the Rdd. (Column 9 and 10)

In [8]:
def changefromnull(autoStr):
  attList = autoStr.split(",")
  if not (attList[9].isdigit()) and not (attList[10].isdigit()):
    attList[9] = 0
    attList[10] = 0
    return attList
  if not (attList[9].isdigit()):
    attList[9] = 0
    attList[10] = int(attList[10])
    return attList
  if not (attList[10].isdigit()):
    attList[9] = int(attList[9])
    attList[10] = 0
    return attList
  else:
    attList[9] = int(attList[9])
    attList[10] = int(attList[10])
    return attList

In [9]:
dataLines1 = dataLines.map(lambda x: changefromnull(x))
dataLines1.take(10)

Out[ 30 ]: 
[['1000001', 'P00069042', 'F', '0-17', '10', 'A', '2', '0', '3', 0, 0, '8370'],
 ['1000001',
 'P00248942',
 'F',
 '0-17',
 '10',
 'A',
 '2',
 '0',
 '1',
 6,
 14,
 '15200'],
 ['1000001',
 'P00087842',
 'F',
 '0-17',
 '10',
 'A',
 '2',
 '0',
 '12',
 0,
 0,
 '1422'],
 ['1000001',
 'P00085442',
 'F',
 '0-17',
 '10',
 'A',
 '2',
 '0',
 '12',
 14,
 0,
 '1057'],
 ['1000002', 'P00285442', 'M', '55+', '16', 'C', '4+', '0', '8', 0, 0, '7969'],
 ['1000003',
 'P00193542',
 'M',
 '26-35',
 '15',
 'A',
 '3',
 '0',
 '1',
 2,
 0,
 '15227'],
 ['1000004',
 'P00184942',
 'M',
 '46-50',
 '7',
 'B',
 '2',
 '1',
 '1',
 8,
 17,
 '19215'],
 ['1000004',
 'P00346142',
 'M',
 '46-50',
 '7',
 'B',
 '2',
 '1',
 '1',
 15,
 0,
 '15854'],
 ['1000004',
 'P0097242',
 'M',
 '46-50',
 '7',
 'B',
 '2',
 '1',
 '1',
 16,
 0,
 '15686'],
 ['1000005',
 'P00274942',
 'M',
 '26-35',
 '20',
 'A',
 '1',
 '1',
 '8',
 0,
 0,
 '7871']]

### Mapping rdd to rows

In [11]:
from pyspark.sql import Row
bfMap = dataLines1.map(lambda p: Row(User_ID = int(p[0]),\
                                  Gender = (p[2]),\
                                  Age = p[3],\
                                  Occupation = int(p[4]),
                                  City_Category = p[5],\
                                  Stay_Current_City_Years = p[6],\
                                  Marital_Status = int(p[7]),\
                                  Product_Category_1 = int(p[8]),\
                                  Product_Category_2 = (p[9]),\
                                  Product_Category_3 = (p[10]),\
                                  Purchase = float(p[11])
                                  ))

In [12]:
bfMap.take(4)

Out[ 10 ]: 
[Row(Age='0-17', City_Category='A', Gender='F', Marital_Status=0, Occupation=10, Product_Category_1=3, Product_Category_2=0, Product_Category_3=0, Purchase=8370.0, Stay_Current_City_Years='2', User_ID=1000001),
 Row(Age='0-17', City_Category='A', Gender='F', Marital_Status=0, Occupation=10, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200.0, Stay_Current_City_Years='2', User_ID=1000001),
 Row(Age='0-17', City_Category='A', Gender='F', Marital_Status=0, Occupation=10, Product_Category_1=12, Product_Category_2=0, Product_Category_3=0, Purchase=1422.0, Stay_Current_City_Years='2', User_ID=1000001),
 Row(Age='0-17', City_Category='A', Gender='F', Marital_Status=0, Occupation=10, Product_Category_1=12, Product_Category_2=14, Product_Category_3=0, Purchase=1057.0, Stay_Current_City_Years='2', User_ID=1000001)]

### Converting Rdd to a DataFrame

In [14]:
bfDf = ss.createDataFrame(bfMap)
bfDf.cache()


Out[ 99 ]: DataFrame[Age: string, City_Category: string, Gender: string, Marital_Status: bigint, Occupation: bigint, Product_Category_1: bigint, Product_Category_2: bigint, Product_Category_3: bigint, Purchase: double, Stay_Current_City_Years: string, User_ID: bigint]

In [15]:
bfDf.show()

+-----+-------------+------+--------------+----------+------------------+------------------+------------------+--------+-----------------------+-------+
 Age|City_Category|Gender|Marital_Status|Occupation|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|Stay_Current_City_Years|User_ID|
+-----+-------------+------+--------------+----------+------------------+------------------+------------------+--------+-----------------------+-------+
 0-17| A| F| 0| 10| 3| 0| 0| 8370.0| 2|1000001|
 0-17| A| F| 0| 10| 1| 6| 14| 15200.0| 2|1000001|
 0-17| A| F| 0| 10| 12| 0| 0| 1422.0| 2|1000001|
 0-17| A| F| 0| 10| 12| 14| 0| 1057.0| 2|1000001|
 55+| C| M| 0| 16| 8| 0| 0| 7969.0| 4+|1000002|
26-35| A| M| 0| 15| 1| 2| 0| 15227.0| 3|1000003|
46-50| B| M| 1| 7| 1| 8| 17| 19215.0| 2|1000004|
46-50| B| M| 1| 7| 1| 15| 0| 15854.0| 2|1000004|
46-50| B| M| 1| 7| 1| 16| 0| 15686.0| 2|1000004|
26-35| A| M| 1| 20| 8| 0| 0| 7871.0| 1|1000005|
26-35| A| M| 1| 20| 5| 11| 0| 5254.0| 1|1000005|
26-35| A| M| 1| 20| 8| 0| 0| 3957.0| 1|1000005|
26-35| A| M| 1| 20| 8| 0| 0| 6073.0| 1|1000005|
26-35| A| M| 1| 20| 1| 2| 5| 15665.0| 1|1000005|
51-55| A| F| 0| 9| 5| 8| 14| 5378.0| 1|1000006|
51-55| A| F| 0| 9| 4| 5| 0| 2079.0| 1|1000006|
51-55| A| F| 0| 9| 2| 3| 4| 13055.0| 1|1000006|
51-55| A| F| 0| 9| 5| 14| 0| 8851.0| 1|1000006|
36-45| B| M| 1| 1| 1| 14| 16| 11788.0| 1|1000007|
26-35| C| M| 1| 12| 1| 5| 15| 19614.0| 4+|1000008|
+-----+-------------+------+--------------+----------+------------------+------------------+------------------+--------+-----------------------+-------+
only showing top 20 rows

In [16]:
bfDf.cache()

Out[ 100 ]: DataFrame[Age: string, City_Category: string, Gender: string, Marital_Status: bigint, Occupation: bigint, Product_Category_1: bigint, Product_Category_2: bigint, Product_Category_3: bigint, Purchase: double, Stay_Current_City_Years: string, User_ID: bigint]

### Indexing categorical data for Modelling.

In [18]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = "City_Category", outputCol="IND_City")
si_model = si.fit(bfDf)
BfNormDf = si_model.transform(bfDf)

In [19]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = "Age", outputCol="IND_Age")
si_model = si.fit(BfNormDf)
BfNormDf = si_model.transform(BfNormDf)

In [20]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = "Gender", outputCol="IND_Gender")
si_model = si.fit(BfNormDf)
BfNormDf = si_model.transform(BfNormDf)

In [21]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = "Stay_Current_City_Years", outputCol="IND_Stay_In_City")
si_model = si.fit(BfNormDf)
BfNormDf = si_model.transform(BfNormDf)

### Correlation information.

In [23]:
for i in BfNormDf.columns:
  if not(isinstance(BfNormDf.select(i).take(1)[0][0],str)):
    print("Correlation to Species for", i,\
         BfNormDf.stat.corr("Purchase",i))

Correlation to Species for Marital_Status 0.00012901814909541888
Correlation to Species for Occupation 0.021104340194127554
Correlation to Species for Product_Category_1 -0.3141247354708834
Correlation to Species for Product_Category_2 0.03839507026749786
Correlation to Species for Product_Category_3 0.2841198836837713
Correlation to Species for Purchase 1.0
Correlation to Species for User_ID 0.005389472267435784
Correlation to Species for IND_City -0.010854036609876973
Correlation to Species for IND_Age 0.002174378755175168
Correlation to Species for IND_Gender -0.0600861659869464
Correlation to Species for IND_Stay_In_City -0.0032702991465657183

### Model Description

In [25]:
BfNormDf.describe().show()

+-------+------+-------------+------+-------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
summary| Age|City_Category|Gender| Marital_Status| Occupation|Product_Category_1|Product_Category_2|Product_Category_3| Purchase|Stay_Current_City_Years| User_ID| IND_City| IND_Age| IND_Gender| IND_Stay_In_City|
+-------+------+-------------+------+-------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
 count|537577| 537577|537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577| 537577|
 mean| null| null| null| 0.4087972513705013|8.082710011775058| 5.295546498455105| 6.784907092379324| 3.871772787898292|9333.859852635065| 1.4692404461960729|1002991.8470284257|0.8477334409768275|1.450162488350506|0.2459126785558162|1.5358730005189953|
 stddev| null| null| null|0.49161215222967514|6.524119769935539|3.7507009498798096| 6.21161846455343| 6.265963443383427|4981.022132656451| 0.9896412335600977|1714.3926953608816| 0.816819144371732| 1.62650944151932|0.4306274236930542|1.4396749857068285|
 min| 0-17| A| F| 0| 0| 1| 0| 0| 185.0| 0| 1000001| 0.0| 0.0| 0.0| 0.0|
 max| 55+| C| M| 1| 20| 18| 18| 18| 23961.0| 4+| 1006040| 2.0| 6.0| 1.0| 4.0|
+-------+------+-------------+------+-------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------------+------------------+------------------+-----------------+------------------+------------------+

Basic Visualisation

In [27]:
BfNormDf.createOrReplaceTempView("BfNormTable")

In [28]:
%sql
select * from BfNormTable

Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_Current_City_Years,User_ID,IND_City,IND_Age,IND_Gender,IND_Stay_In_City
0-17,A,F,0,10,3,0,0,8370.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,1,6,14,15200.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,0,0,1422.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,14,0,1057.0,2,1000001,2.0,6.0,1.0,1.0
55+,C,M,0,16,8,0,0,7969.0,4+,1000002,1.0,5.0,0.0,3.0
26-35,A,M,0,15,1,2,0,15227.0,3,1000003,2.0,0.0,0.0,2.0
46-50,B,M,1,7,1,8,17,19215.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,15,0,15854.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,16,0,15686.0,2,1000004,0.0,3.0,0.0,1.0
26-35,A,M,1,20,8,0,0,7871.0,1,1000005,2.0,0.0,0.0,0.0


In [29]:
%sql
select * from BfNormTable

Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_Current_City_Years,User_ID,IND_City,IND_Age,IND_Gender,IND_Stay_In_City
0-17,A,F,0,10,3,0,0,8370.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,1,6,14,15200.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,0,0,1422.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,14,0,1057.0,2,1000001,2.0,6.0,1.0,1.0
55+,C,M,0,16,8,0,0,7969.0,4+,1000002,1.0,5.0,0.0,3.0
26-35,A,M,0,15,1,2,0,15227.0,3,1000003,2.0,0.0,0.0,2.0
46-50,B,M,1,7,1,8,17,19215.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,15,0,15854.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,16,0,15686.0,2,1000004,0.0,3.0,0.0,1.0
26-35,A,M,1,20,8,0,0,7871.0,1,1000005,2.0,0.0,0.0,0.0


In [30]:
%sql
select * from BfNormTable

Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_Current_City_Years,User_ID,IND_City,IND_Age,IND_Gender,IND_Stay_In_City
0-17,A,F,0,10,3,0,0,8370.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,1,6,14,15200.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,0,0,1422.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,14,0,1057.0,2,1000001,2.0,6.0,1.0,1.0
55+,C,M,0,16,8,0,0,7969.0,4+,1000002,1.0,5.0,0.0,3.0
26-35,A,M,0,15,1,2,0,15227.0,3,1000003,2.0,0.0,0.0,2.0
46-50,B,M,1,7,1,8,17,19215.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,15,0,15854.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,16,0,15686.0,2,1000004,0.0,3.0,0.0,1.0
26-35,A,M,1,20,8,0,0,7871.0,1,1000005,2.0,0.0,0.0,0.0


In [31]:
%sql
select * from BfNormTable

Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_Current_City_Years,User_ID,IND_City,IND_Age,IND_Gender,IND_Stay_In_City
0-17,A,F,0,10,3,0,0,8370.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,1,6,14,15200.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,0,0,1422.0,2,1000001,2.0,6.0,1.0,1.0
0-17,A,F,0,10,12,14,0,1057.0,2,1000001,2.0,6.0,1.0,1.0
55+,C,M,0,16,8,0,0,7969.0,4+,1000002,1.0,5.0,0.0,3.0
26-35,A,M,0,15,1,2,0,15227.0,3,1000003,2.0,0.0,0.0,2.0
46-50,B,M,1,7,1,8,17,19215.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,15,0,15854.0,2,1000004,0.0,3.0,0.0,1.0
46-50,B,M,1,7,1,16,0,15686.0,2,1000004,0.0,3.0,0.0,1.0
26-35,A,M,1,20,8,0,0,7871.0,1,1000005,2.0,0.0,0.0,0.0


### Creating Labeled points from labels to features.

In [33]:
from pyspark.ml.linalg import Vectors
def transformToLabeledPoint(row) :
    lp = ( row["Purchase"], Vectors.dense([row["IND_Age"],\
                        row["IND_Gender"], \
                        row["IND_Stay_In_City"], \
                        row["Product_Category_1"],\
                        row["Product_Category_2"],\
                        row["Product_Category_3"],
                        row["Marital_Status"],
                        row["Occupation"]]))
    return lp
    

In [34]:
bfLabPoint = BfNormDf.rdd.map(transformToLabeledPoint)
bfDf = ss.createDataFrame(bfLabPoint,["label","features"])
bfDf.select("label","features").show(1000)

+-------+--------------------+
 label| features|
+-------+--------------------+
 8370.0|[6.0,1.0,1.0,3.0,...|
15200.0|[6.0,1.0,1.0,1.0,...|
 1422.0|[6.0,1.0,1.0,12.0...|
 1057.0|[6.0,1.0,1.0,12.0...|
 7969.0|[5.0,0.0,3.0,8.0,...|
15227.0|[0.0,0.0,2.0,1.0,...|
19215.0|[3.0,0.0,1.0,1.0,...|
15854.0|[3.0,0.0,1.0,1.0,...|
15686.0|[3.0,0.0,1.0,1.0,...|
 7871.0|[0.0,0.0,0.0,8.0,...|
 5254.0|[0.0,0.0,0.0,5.0,...|
 3957.0|[0.0,0.0,0.0,8.0,...|
 6073.0|[0.0,0.0,0.0,8.0,...|
15665.0|[0.0,0.0,0.0,1.0,...|
 5378.0|[4.0,1.0,0.0,5.0,...|
 2079.0|[4.0,1.0,0.0,4.0,...|
13055.0|[4.0,1.0,0.0,2.0,...|
 8851.0|[4.0,1.0,0.0,5.0,...|
11788.0|[1.0,0.0,0.0,1.0,...|
19614.0|[0.0,0.0,3.0,1.0,...|
 8584.0|[0.0,0.0,3.0,5.0,...|
 9872.0|[0.0,0.0,3.0,8.0,...|
 9743.0|[0.0,0.0,3.0,8.0,...|
 5982.0|[0.0,0.0,3.0,8.0,...|
11927.0|[0.0,0.0,3.0,1.0,...|
16662.0|[0.0,0.0,4.0,6.0,...|
 5887.0|[0.0,0.0,4.0,8.0,...|
 6973.0|[0.0,0.0,4.0,5.0,...|
 5391.0|[0.0,0.0,4.0,5.0,...|
16352.0|[1.0,1.0,3.0,2.0,...|
 8886.0|[1.0,1.0,3.0,5.0,...|
 5875.0|[1.0,1.0,3.0,8.0,...|
 8854.0|[1.0,1.0,3.0,5.0,...|
10946.0|[1.0,1.0,3.0,3.0,...|
 5152.0|[1.0,1.0,3.0,5.0,...|
 7089.0|[1.0,1.0,3.0,5.0,...|
12909.0|[1.0,1.0,3.0,2.0,...|
 8770.0|[1.0,1.0,3.0,5.0,...|
15212.0|[1.0,1.0,3.0,1.0,...|
15705.0|[1.0,1.0,3.0,1.0,...|
 7947.0|[1.0,1.0,3.0,14.0...|
18963.0|[1.0,1.0,3.0,1.0,...|
 8718.0|[1.0,1.0,3.0,5.0,...|
16406.0|[1.0,1.0,3.0,2.0,...|
12642.0|[1.0,1.0,3.0,6.0,...|
10007.0|[1.0,1.0,3.0,8.0,...|
11562.0|[1.0,1.0,3.0,1.0,...|
 6171.0|[0.0,1.0,0.0,8.0,...|
19327.0|[0.0,1.0,0.0,1.0,...|
 8027.0|[0.0,1.0,0.0,8.0,...|
15246.0|[0.0,0.0,1.0,1.0,...|
 6865.0|[0.0,0.0,1.0,5.0,...|
 5839.0|[3.0,0.0,2.0,8.0,...|
15912.0|[3.0,0.0,2.0,1.0,...|
12107.0|[3.0,0.0,2.0,1.0,...|
 5848.0|[1.0,0.0,4.0,8.0,...|
19653.0|[0.0,0.0,0.0,1.0,...|
 5958.0|[0.0,0.0,0.0,8.0,...|
11415.0|[0.0,0.0,0.0,1.0,...|
 5380.0|[0.0,0.0,0.0,5.0,...|
 3594.0|[0.0,0.0,0.0,5.0,...|
 4209.0|[0.0,0.0,0.0,8.0,...|
 5407.0|[0.0,0.0,0.0,5.0,...|
 7803.0|[0.0,0.0,0.0,8.0,...|
11458.0|[0.0,0.0,0.0,1.0,...|
11755.0|[1.0,1.0,4.0,1.0,...|
 8702.0|[1.0,1.0,4.0,5.0,...|
15872.0|[4.0,0.0,4.0,1.0,...|
 8609.0|[4.0,0.0,4.0,5.0,...|
15172.0|[4.0,0.0,4.0,1.0,...|
 1780.0|[2.0,1.0,2.0,5.0,...|
10754.0|[2.0,1.0,2.0,3.0,...|
 2802.0|[2.0,1.0,2.0,4.0,...|
19473.0|[2.0,1.0,2.0,1.0,...|
19672.0|[2.0,1.0,2.0,1.0,...|
19313.0|[2.0,1.0,2.0,1.0,...|
15182.0|[2.0,1.0,2.0,1.0,...|
15985.0|[2.0,1.0,2.0,2.0,...|
16454.0|[2.0,1.0,2.0,2.0,...|
13551.0|[2.0,1.0,2.0,3.0,...|
 697.0|[2.0,1.0,2.0,4.0,...|
 7030.0|[2.0,1.0,2.0,5.0,...|
 5430.0|[2.0,1.0,2.0,5.0,...|
 8011.0|[2.0,1.0,2.0,8.0,...|
 5912.0|[2.0,1.0,2.0,8.0,...|
 7746.0|[6.0,0.0,2.0,1.0,...|
19525.0|[6.0,0.0,2.0,1.0,...|
11959.0|[6.0,0.0,2.0,1.0,...|
19172.0|[6.0,0.0,2.0,1.0,...|
 8773.0|[6.0,0.0,2.0,5.0,...|
11023.0|[6.0,0.0,2.0,3.0,...|
 4654.0|[6.0,0.0,2.0,11.0...|
 4664.0|[6.0,0.0,2.0,11.0...|
 8290.0|[6.0,0.0,2.0,3.0,...|
 7011.0|[6.0,0.0,2.0,5.0,...|
 4211.0|[6.0,0.0,2.0,1.0,...|
 1539.0|[6.0,0.0,2.0,11.0...|
 3055.0|[2.0,0.0,4.0,3.0,...|
12099.0|[2.0,0.0,3.0,1.0,...|
 8797.0|[2.0,0.0,3.0,5.0,...|
10681.0|[2.0,0.0,3.0,3.0,...|
 7801.0|[2.0,0.0,3.0,8.0,...|
15517.0|[2.0,0.0,3.0,1.0,...|
12098.0|[2.0,0.0,3.0,1.0,...|
 6976.0|[2.0,0.0,3.0,5.0,...|
 5384.0|[2.0,0.0,3.0,5.0,...|
 6973.0|[2.0,0.0,3.0,5.0,...|
 5168.0|[2.0,0.0,3.0,5.0,...|
 3449.0|[2.0,0.0,3.0,5.0,...|
 7177.0|[2.0,0.0,3.0,5.0,...|
 4570.0|[2.0,0.0,3.0,11.0...|
 9564.0|[2.0,0.0,3.0,2.0,...|
15766.0|[1.0,0.0,2.0,1.0,...|
 8481.0|[1.0,0.0,2.0,6.0,...|
 6108.0|[1.0,0.0,2.0,11.0...|
15354.0|[1.0,0.0,2.0,1.0,...|
12015.0|[1.0,0.0,2.0,1.0,...|
15371.0|[1.0,0.0,2.0,1.0,...|
19239.0|[1.0,0.0,2.0,1.0,...|
16137.0|[1.0,0.0,2.0,2.0,...|
11887.0|[1.0,0.0,2.0,1.0,...|
 5183.0|[1.0,0.0,2.0,5.0,...|
 584.0|[1.0,0.0,2.0,13.0...|
 7601.0|[1.0,0.0,2.0,1.0,...|
 2230.0|[0.0,1.0,2.0,8.0,...|
 6940.0|[0.0,1.0,2.0,5.0,...|
15774.0|[0.0,1.0,2.0,1.0,...|
15361.0|[2.0,0.0,3.0,1.0,...|
15770.0|[2.0,0.0,3.0,1.0,...|
18974.0|[2.0,0.0,3.0,1.0,...|
 1848.0|[0.0,0.0,1.0,5

In [35]:
bfDf.cache()

Out[ 81 ]: DataFrame[label: double, features: vector]

### Splitting into test and training data

In [37]:
(trainingData, testData) = bfDf.randomSplit([0.8, 0.2])
trainingData.count()

Out[ 82 ]: 430181

In [38]:
testData.count()

Out[ 83 ]: 107396

### Fitting Linear Model

In [40]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=100)
lrModel = lr.fit(trainingData)

### Model coefficients and intercepts

In [42]:
#Print the metrics
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [26.8574807724,-483.069544723,-11.247847746,-317.256183484,8.44009817165,149.790927196,50.5171987611,9.6449033047]
Intercept: 10370.859966820502

### Predictions

In [44]:
#Predict on the test data
predictions = lrModel.transform(testData)
predictions.show(50000)

+-------+--------------------+------------------+
 label| features| prediction|
+-------+--------------------+------------------+
 186.0|[1.0,1.0,1.0,13.0...| 6003.208052556154|
 186.0|[3.0,0.0,0.0,13.0...| 9034.816299477914|
 187.0|[3.0,0.0,2.0,13.0...| 6605.90393751503|
 187.0|[5.0,0.0,4.0,13.0...| 9014.60516012452|
 189.0|[0.0,0.0,0.0,13.0...| 6574.469218366283|
 201.0|[0.0,0.0,1.0,13.0...| 8868.1294652147|
 218.0|[0.0,0.0,0.0,13.0...| 6410.492937705752|
 218.0|[1.0,0.0,0.0,13.0...| 6524.167472701129|
 220.0|[0.0,0.0,3.0,13.0...| 6453.921545385905|
 222.0|[0.0,0.0,1.0,13.0...| 6437.837627659191|
 347.0|[2.0,1.0,0.0,12.0...| 6389.913884738986|
 348.0|[3.0,0.0,0.0,12.0...| 6694.8754060885|
 354.0|[1.0,0.0,1.0,12.0...|6714.4240443022445|
 358.0|[3.0,0.0,4.0,12.0...| 6796.980099421584|
 361.0|[0.0,0.0,2.0,12.0...| 6726.965767054049|
 363.0|[0.0,0.0,3.0,12.0...| 6831.456758964374|
 370.0|[2.0,0.0,1.0,13.0...| 6559.066912336904|
 371.0|[0.0,0.0,0.0,13.0...| 8962.68395533231|
 371.0|[0.0,0.0,1.0,13.0...| 6440.130309896843|
 371.0|[0.0,0.0,3.0,13.0...| 6376.762318948333|
 371.0|[1.0,0.0,0.0,13.0...| 6475.942956177645|
 372.0|[0.0,0.0,4.0,13.0...| 8735.644206691975|
 372.0|[2.0,0.0,0.0,13.0...| 6485.803312578322|
 375.0|[0.0,0.0,0.0,13.0...| 6538.182287385148|
 375.0|[0.0,1.0,2.0,13.0...| 5876.005912057776|
 375.0|[0.0,1.0,2.0,13.0...| 5895.295718667169|
 375.0|[1.0,1.0,4.0,13.0...| 5899.657503947517|
 376.0|[1.0,0.0,1.0,12.0...| 6627.619914559975|
 378.0|[0.0,0.0,2.0,13.0...| 8883.523645145102|
 378.0|[1.0,1.0,3.0,13.0...| 8395.569121706547|
 379.0|[2.0,0.0,0.0,13.0...| 6465.425628864321|
 380.0|[0.0,1.0,1.0,13.0...| 8346.480307273414|
 380.0|[1.0,0.0,4.0,13.0...| 8905.9704319018|
 381.0|[0.0,0.0,0.0,13.0...| 8850.442603046647|
 382.0|[0.0,1.0,0.0,13.0...| 8357.72815501945|
 382.0|[1.0,0.0,2.0,13.0...| 6503.964459446708|
 382.0|[2.0,0.0,0.0,13.0...| 6389.341355050668|
 383.0|[0.0,0.0,1.0,13.0...| 8942.996009414623|
 383.0|[1.0,1.0,3.0,12.0...| 6346.309985100201|
 384.0|[1.0,0.0,3.0,13.0...| 6454.136998481887|
 385.0|[0.0,1.0,0.0,13.0...| 5949.018806310984|
 385.0|[1.0,0.0,0.0,13.0...| 6533.812376005825|
 386.0|[0.0,0.0,2.0,13.0...|6523.0388129601215|
 386.0|[0.0,0.0,3.0,13.0...| 6280.300361420678|
 386.0|[2.0,0.0,1.0,13.0...| 6424.025341590464|
 387.0|[2.0,1.0,0.0,12.0...| 6173.010795051232|
 389.0|[0.0,1.0,1.0,13.0...| 5995.640378393128|
 389.0|[2.0,1.0,1.0,13.0...| 5844.506763820997|
 390.0|[1.0,0.0,0.0,13.0...| 6379.480998649991|
 390.0|[1.0,0.0,1.0,13.0...| 6426.115495212824|
 392.0|[0.0,1.0,0.0,13.0...| 5823.622138869239|
 392.0|[0.0,1.0,0.0,13.0...| 6033.530253815601|
 393.0|[5.0,0.0,0.0,13.0...| 6720.694207770726|
 394.0|[0.0,0.0,3.0,13.0...| 6533.373454061131|
 396.0|[2.0,1.0,2.0,13.0...| 5968.300486821401|
 397.0|[0.0,0.0,4.0,13.0...| 8870.672852957729|
 398.0|[0.0,0.0,0.0,13.0...| 6488.869893757057|
 399.0|[3.0,0.0,0.0,13.0...| 9093.773596410703|
 399.0|[5.0,0.0,3.0,13.0...| 6688.155469665663|
 400.0|[0.0,0.0,0.0,12.0...| 6742.109241479076|
 400.0|[0.0,0.0,3.0,13.0...| 6531.080771823479|
 402.0|[0.0,0.0,0.0,13.0...| 8858.8827012183|
 402.0|[0.0,0.0,2.0,13.0...|6428.8824621508065|
 403.0|[0.0,0.0,0.0,13.0...| 8811.86298982786|
 403.0|[2.0,0.0,3.0,12.0...|6641.6266031450195|
 404.0|[2.0,0.0,3.0,13.0...| 6382.239839488999|
 404.0|[2.0,0.0,4.0,13.0...| 6574.752690755325|
 405.0|[2.0,0.0,4.0,12.0...| 6582.154238875501|
 407.0|[2.0,1.0,4.0,13.0...| 5945.80479132933|
 555.0|[1.0,0.0,0.0,13.0...| 6331.256482126508|
 555.0|[1.0,1.0,2.0,13.0...| 5989.667522572465|
 555.0|[3.0,1.0,0.0,13.0...| 8367.288786833133|
 555.0|[4.0,0.0,3.0,13.0...| 6580.641275084975|
 556.0|[0.0,1.0,3.0,13.0...| 6059.948812643326|
 556.0|[2.0,0.0,4.0,13.0...| 6554.258079012887|
 556.0|[3.0,0.0,0.0,13.0...| 8968.389853449646|
 556.0|[3.0,0.0,0.0,13.0...| 6705.558859444674|
 556.0|[3.0,0.0,4.0,13.0...| 8904.108655856111|
 557.0|[0.0,0.0,1.0,13.0...| 6536.579342943809|
 557.0|[4.0,0.0,0.0,13.0...| 6556.515398494902|
 558.0|[0.0,0.0,0.0,13.0...| 6256.1744848306

### Model accuracy using root mean square error.

In [46]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
evaluator.evaluate(predictions)

Out[ 105 ]: 0.13130626658771227

# Classification attempt!

In [48]:
%sql
CREATE TEMP VIEW New_Black_Friday
AS SELECT 
* FROM
(
SELECT 
    T1.User_ID,
    0 Classification
FROM BfNormTable T1
) 

In [49]:
%sql
Drop view New_Black_Friday

In [50]:
%sql
SELECT User_ID,Purchase
from BFNormTable
order by Purchase asc

User_ID,Purchase
1004227,185.0
1004048,185.0
1001968,185.0
1003391,185.0
1000889,186.0
1005039,186.0
1006025,186.0
1001630,186.0
1001198,187.0
1005120,187.0


In [51]:
%sql
create temp view newdf 
as
(select a.User_ID,a.Classification, b.Age,b.City_Category,b.Gender,b.Marital_Status,b.Occupation,b.Product_Category_1,b.Product_Category_2,b.Product_Category_3, b.Purchase, b.Stay_Current_City_Years, b.IND_City, b.IND_Age, b.IND_Gender, b.IND_Stay_In_City 
from 
New_Black_Friday a
LEFT JOIN BfNormTable b
ON a.User_ID = b.USER_ID)



In [52]:
blackFridayDf = sqlContext.table("select * from newdf")

--------------------------------------------------------------------------- 
 ParseException Traceback (most recent call last)
 <command-3814192289793238> in <module> () 
 ----> 1 blackFridayDf = sqlContext . table ( "select * from newdf" ) 

 /databricks/spark/python/pyspark/sql/context.py in table (self, tableName) 
 369 True 
 370 """
 --> 371 return self . sparkSession . table ( tableName ) 
 372 
 373 @ ignore_unicode_prefix 

 /databricks/spark/python/pyspark/sql/session.py in table (self, tableName) 
 840 True 
 841 """
 --> 842 return DataFrame ( self . _jsparkSession . table ( tableName ) , self . _wrapped ) 
 843 
 844 @ property 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nmismatched input '*' expecting <EOF>(line 1, pos 7)\n\n== SQL ==\nselect * from newdf\n-------^^^\n"

In [53]:
%sql
select * from newdf

User_ID,Classification,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_Current_City_Years,IND_City,IND_Age,IND_Gender,IND_Stay_In_City
1000012,0,26-35,C,M,0,12,1,6,0,15246.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,5,8,0,6865.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,1,6,8,19031.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,5,0,0,5196.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,10,13,16,14249.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,5,6,8,5464.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,5,0,0,3576.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,6,0,0,16599.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,8,16,0,9696.0,2,1.0,0.0,0.0,1.0
1000012,0,26-35,C,M,0,12,1,8,17,12091.0,2,1.0,0.0,0.0,1.0
